## Chapter 7　実装

---
### 7.1　GPUとGPGPU
画像に対するGPU利用と比べて自然言語処理分野ではGPU利用が難しかったが、  
近年GPUを利用するための手法が発展してきた。

GPU利用には計算の並列化が重要であり、その代表はミニバッチ化である。

---
### 7.2　RNNにおけるミニバッチ化
自然言語処理においてミニバッチ化を妨げていた問題は  
入力文や出力文の長さが一定でないということである。

古くから文長ごとにデータセットを分割する手法はあったが、これは無作為抽出を不可能にするため  
モデルの予測能力に深刻な悪影響を与えることがある。

そこで考えられたのが padding を文へ適用することで、  
nullなど事前に定めた特殊な単語を付加することで文長を揃え並列計算を効率化する。    
ただし、全ての文がミニバッチ中の最大長に揃えられることになるため、計算量そのものは増大する。

また、文長によりソートすることで並列計算における無駄な分岐を減らすことができるので、  
これも計算速度向上に寄与する。  

paddngによる計算量増大を軽減するためにミニバッチごとに文長を選択して  
それぞれの文長に合わせた計算グラフを事前に準備することで計算時間を減らすテクニックがあり、  
これをバケッティング（bucketing）という。  
ただし、文長の種類が多いと計算グラフを事前準備する意味は失われていく。

---
#### 多層のRNNの高速化
RNNにおいて時刻 $t$ における $k$ 番目の層の出力を $\mathbf{h}_{t}^{(k)}$ と表記する。  
時刻 $t$ または層の番号 $k$ が一致する２つの出力は依存性が存在するために並列計算できないが、  
$t$ と $k$ の両方が異なれば依存関係が存在しないため、理論的には並列計算可能である。

J.Appleyard et al. [Optimizing Performance of Recurrent Neural Networks on GPUs](https://arxiv.org/abs/1604.01946) arXiv:1604.01946 [cs.LG] 2016.  
ではRNNに種々の計算効率改善策を実装し、最大11倍の高速化を果たしたと報告されている。

---
#### 持続性再帰ニューラルネット （persistent RNN）
RNNの隠れ状態ベクトルを $\mathbf{h} \in \mathbb{R}^{D}$、バッチ数を $B$ とする。  
このとき隠れ状態ベクトルとパラメータ行列の積の時間計算量は $O(BD^{2})$、  
GPUにおけるメモリの消費量は $O(D^{2})$ である。

計算中プロセッサ上のレジスタに行列を保持して読み込み時間を抑える  
持続性再帰ニューラルネットという手法が提案されている。

---
#### 木構造再帰ニューラルネットのミニバッチ化
Tree-RNNの単純なミニバッチ化は困難である。

再帰的な伝播計算を、スタックを用いる直列的な計算に変換することを考える。  
各ノードに木の帰りがけ順に番号を振り、番号の小さい順にたどりながら処理を行うこととする。

葉ノードに対して埋め込みベクトルを計算する操作を $\Lambda$、  
各ノードに対して２つの子ノードの隠れ状態ベクトル $\mathbf{h}_{L}, \mathbf{h}_{R}$ から  
親ノードの隠れ状態ベクトル $\mathbf{h}_{P}$ を計算する操作を $\Psi$ とする。

帰りがけ順にノードをたどるとき、葉ノードで $\Lambda$ を、それ以外のノードで $\Psi$ を行えばよい。  
この操作はスタックを利用した木構造の走査と見なすことができるので、  
$\Lambda$ では計算結果をプッシュし、 $\Psi$ ではポップ操作で２データを読み込み計算結果をプッシュする。

各ステップでどの変数に $\Lambda, \Psi$ いずれの操作を行うかがわかれば並列化可能である。  
これは上記のようなスタックの情報 $\{ \mathbf{h}_{i} \}$ を保持すれば可能である。

文長 $I$ について $\{\mathbf{h}_{i}\}$ の大きさは $2I-1$、各要素は $\mathbf{h}_{i} \in \mathbb{R}^{D}$ であり、  
各ステップごとに複製が必要であるから、前向き計算で $O(I^{2}D)$ のメモリを消費する。  
これはシンスタック（thin stack）により空間計算量を $O(ID)$ まで減らすことができる。

---
### 7.3　無作為抽出
